# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [2]:
# Import necessary libraries
import os
import requests
import json
from typing import List, Dict
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin

# Load environment variables from a .env file
load_dotenv()

# Define constants
MODEL = 'gpt-4o-mini'  # Specify the OpenAI model to use
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')  # Get API key from environment or use default

# Initialize OpenAI client with the API key
openai = OpenAI(api_key=OPENAI_API_KEY)

class Website:
    """
    A class to represent a website and its contents.
    """
    def __init__(self, url: str):
        """
        Initialize the Website object with a given URL.
        
        :param url: The URL of the website to scrape
        """
        self.url = url
        self.title, self.text, self.links = self._scrape_website()

    def _scrape_website(self) -> tuple:
        """
        Scrape the website content, extracting title, text, and links.
        
        :return: A tuple containing the title, text content, and links of the website
        """
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.title.string if soup.title else "No title found"
        
        # Extract text content
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()  # Remove unwanted tags
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        
        # Extract links
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        
        return title, text, links

    def get_contents(self) -> str:
        """
        Get a formatted string of the website contents.
        
        :return: A string containing the website title and text content
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

class LinkAnalyzer:
    """
    A class to analyze and categorize links from a website.
    """
    # System prompt for the OpenAI model to categorize links
    LINK_SYSTEM_PROMPT = """
    You are provided with a list of links found on a webpage. Your task is to first categorize each link into one of the following categories:
    - about page
    - careers page
    - terms of service
    - privacy policy
    - contact page
    - other (please specify).

    Once the links are categorized, please choose which links are most relevant to include in a brochure about the company. 
    The brochure should only include links such as About pages, Careers pages, or Company Overview pages. Exclude any links related to Terms of Service, Privacy Policy, or email addresses.

    Respond in the following JSON format:
    {
        "categorized_links": [
            {"category": "about page", "url": "https://full.url/about"},
            {"category": "careers page", "url": "https://full.url/careers"},
            {"category": "terms of service", "url": "https://full.url/terms"},
            {"category": "privacy policy", "url": "https://full.url/privacy"},
            {"category": "other", "specify": "contact page", "url": "https://full.url/contact"}
        ],
        "brochure_links": [
            {"type": "about page", "url": "https://full.url/about"},
            {"type": "careers page", "url": "https://full.url/careers"}
        ]
    }

    Please find the links below and proceed with the task:

    Links (some may be relative links):
    [INSERT LINK LIST HERE]
    """

    @staticmethod
    def get_links(website: Website) -> Dict:
        """
        Analyze and categorize links from a given website.
        
        :param website: A Website object containing the links to analyze
        :return: A dictionary containing categorized links and brochure-relevant links
        """
        # Prepare the user prompt for the OpenAI model
        user_prompt = f"Here is the list of links on the website of {website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)

        # Make an API call to OpenAI for link analysis
        completion = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": LinkAnalyzer.LINK_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)

class BrochureGenerator:
    """
    A class to generate a company brochure based on website content.
    """
    # System prompt for the OpenAI model to generate the brochure
    SYSTEM_PROMPT = """
    You are an assistant that analyzes the contents of several relevant pages from a company website 
    and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.
    Include details of company culture, customers and careers/jobs if you have the information.
    Structure the brochure to include specific sections as follows:
    About Us
    What we do
    How We Do It
    Where We Do It
    Our People
    Our Culture
    Connect with Us.
    Please provide two versions of the brochure, the first in English, the second in Spanish. The contents of the brochure are to be the same for both languages.
    """

    @staticmethod
    def get_all_details(url: str) -> str:
        """
        Gather all relevant details from a company's website.
        
        :param url: The URL of the company's main page
        :return: A string containing all relevant website content
        """
        result = "Landing page:\n"
        website = Website(url)
        result += website.get_contents()

        # Analyze links and get brochure-relevant ones
        links = LinkAnalyzer.get_links(website)
        brochure_links = links.get('brochure_links', [])
        print("Found Brochure links:", brochure_links)

        # Gather content from brochure-relevant pages
        for link in brochure_links:
            result += f"\n\n{link['type']}:\n"
            full_url = urljoin(url, link["url"])
            result += Website(full_url).get_contents()

        return result

    @staticmethod
    def get_brochure_user_prompt(company_name: str, url: str) -> str:
        """
        Generate a user prompt for the OpenAI model to create a brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        :return: A string containing the user prompt for brochure generation
        """
        user_prompt = f"You are looking at a company called: {company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += BrochureGenerator.get_all_details(url)
        return user_prompt[:20_000]  # Truncate if more than 20,000 characters

    @staticmethod
    def stream_brochure(company_name: str, url: str):
        """
        Generate and stream a company brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        """
        # Make a streaming API call to OpenAI for brochure generation
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": BrochureGenerator.SYSTEM_PROMPT},
                {"role": "user", "content": BrochureGenerator.get_brochure_user_prompt(company_name, url)}
            ],
            stream=True
        )

        # Display the generated brochure in real-time
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


In [4]:
# Main execution block
if __name__ == "__main__":
    # Generate a brochure
    BrochureGenerator.stream_brochure("Edward Donner", "https://edwarddonner.com/")

Found Brochure links: [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'other', 'specify': 'Outsmart page', 'url': 'https://edwarddonner.com/outsmart/'}]



# Edward Donner Brochure

## About Us
At Edward Donner, we are committed to revolutionizing the way people connect with career opportunities. Founded by Ed, the co-founder and CTO of Nebula.io, we leverage cutting-edge Generative AI and machine learning to assist recruiters in sourcing, understanding, engaging, and managing talent more effectively than ever before.

## What We Do
We’ve developed a patented matching model that connects candidates with their ideal roles—no keywords necessary. With our innovative approach, we aim to help individuals discover their potential and pursue their passions, leading to higher levels of human prosperity.

## How We Do It
Our award-winning platform uses advanced AI technology, honing in on the unique skills and potentials of jobseekers. We are inspired by the concept of “Ikigai,” which drives our mission to match people with roles that fulfill their career aspirations.

## Where We Do It
Our operations are primarily based in New York City, where we embrace an environment that fosters creativity, innovation, and collaborative spirit. While we are grounded in NYC, our reach extends globally as we work with clients and users from around the world.

## Our People
At Edward Donner, we believe our greatest asset is our talented team. We are composed of dedicated professionals who are experts in software engineering, data science, and technology leadership, all with a shared passion for harnessing AI to solve real-world problems. Our diverse backgrounds contribute to a culture of inclusion and excellence.

## Our Culture
We pride ourselves on cultivating a workplace that thrives on collaboration, openness, and continuous learning. Our work culture emphasizes innovation while also recognizing the importance of personal connections and networking. We encourage our team and connect with others not just virtually, but over coffee when possible!

## Connect with Us
Interested in learning more about what we do? We’d love to hear from you! Whether you’re a potential customer, investor, or recruit, let’s connect!

- **Email:** ed@edwarddonner.com
- **Website:** [www.edwarddonner.com](http://www.edwarddonner.com)
- **Follow Us:**
  - [LinkedIn](https://www.linkedin.com)
  - [Twitter](https://www.twitter.com)
  - [Facebook](https://www.facebook.com)
- **Subscribe to Our Newsletter!**
  
---

# Folleto de Edward Donner

## Acerca de Nosotros
En Edward Donner, estamos comprometidos a revolucionar la forma en que las personas se conectan con oportunidades de carrera. Fundado por Ed, el cofundador y CTO de Nebula.io, aprovechamos la inteligencia artificial generativa y el aprendizaje automático de vanguardia para ayudar a los reclutadores a buscar, comprender, involucrar y gestionar talento de manera más eficaz que nunca.

## Qué Hacemos
Hemos desarrollado un modelo de emparejamiento patentado que conecta a los candidatos con sus roles ideales, sin necesidad de palabras clave. Con nuestro enfoque innovador, buscamos ayudar a las personas a descubrir su potencial y seguir sus pasiones, llevando a niveles más altos de prosperidad humana.

## Cómo Lo Hacemos
Nuestra plataforma galardonada utiliza tecnología avanzada de inteligencia artificial, centrándose en las habilidades y los potenciales únicos de los buscadores de empleo. Nos inspira el concepto de "Ikigai", que impulsa nuestra misión de emparejar a las personas con roles que cumplen sus aspiraciones profesionales.

## Dónde Lo Hacemos
Nuestras operaciones se basan principalmente en la ciudad de Nueva York, donde abrazamos un entorno que fomenta la creatividad, la innovación y el espíritu colaborativo. Si bien estamos enraizados en Nueva York, nuestro alcance se extiende globalmente mientras trabajamos con clientes y usuarios de todo el mundo.

## Nuestro Personal
En Edward Donner, creemos que nuestro mayor activo es nuestro talentoso equipo. Estamos compuestos por profesionales dedicados que son expertos en ingeniería de software, ciencia de datos y liderazgo tecnológico, todos con una pasión compartida por aprovechar la inteligencia artificial para resolver problemas del mundo real. Nuestros diversos antecedentes contribuyen a una cultura de inclusión y excelencia.

## Nuestra Cultura
Nos enorgullecemos de cultivar un lugar de trabajo que prospera en colaboración, apertura y aprendizaje continuo. Nuestra cultura laboral enfatiza la innovación, mientras que también reconoce la importancia de las conexiones personales y el networking. Fomentamos a nuestro equipo y conectamos con otros no solo de forma virtual, ¡sino también tomando un café cuando sea posible!

## Conéctate con Nosotros
¿Interesado en aprender más sobre lo que hacemos? ¡Nos encantaría saber de ti! Ya seas un cliente potencial, inversionista o recluta, ¡conectémonos!

- **Correo Electrónico:** ed@edwarddonner.com
- **Sitio Web:** [www.edwarddonner.com](http://www.edwarddonner.com)
- **Síguenos:**
  - [LinkedIn](https://www.linkedin.com)
  - [Twitter](https://www.twitter.com)
  - [Facebook](https://www.facebook.com)
- **¡Suscríbete a Nuestro Boletín!**

